In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import random
import torch
import torch.backends.cudnn as cudnn
from sklearn.neural_network import MLPRegressor

os.chdir('Resources/')

import warnings
warnings.filterwarnings("ignore")

In [2]:
def power(b, p, m):
    b %= m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        t = m
        m = a % m
        a = t
        t = y
        y = x - q * y
        x = t
    if x < 0:
        x += origin_m
    return x

def encode_signed(m, p):
    return m + 9

def decode_signed(m_encoded, p):
    return m_encoded - (9 * 9)

def encrypt_additive(m, h, g, p, y=7):
    m_enc = encode_signed(m, p)
    c1 = power(g, y, p)
    s = power(h, y, p)
    c2 = (power(g, m_enc, p) * s) % p
    return c1, c2

def decrypt_additive(ciphertext, x, p, g):
    c1, c2 = ciphertext
    s = power(c1, x, p)
    s_inv = mod_inv(s, p)
    m_encoded = (c2 * s_inv) % p
    for m in range(p - 1):
        if power(g, m, p) == m_encoded:
            return decode_signed(m, p)
    return None

In [3]:
def reset_seeds(seed=42):
    import os
    import random
    import numpy as np
    import tensorflow as tf
    import torch
    import torch.backends.cudnn as cudnn

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    torch.manual_seed(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

def preprocess_dataset(df, seed=42):
    reset_seeds()

    X = df[['Plant_ID', 'Machine_Type', 'Quality_Audit', 'Year', 'Month', 'Week']].values
    y = df['Weekly_Production'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    return X, X_test, y, y_test


def build_mlp(input_dim):
    reset_seeds()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mse')
    return model

def train_local(X_train, X_test, y_train, y_test):
    reset_seeds()
    model = build_mlp(X_train.shape[1])
    model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=0)
    y_pred = model.predict(X_test)
    return model, r2_score(y_test, y_pred)

def get_gradients_and_flatten(model, X_train, y_train):
    mse_loss_fn = tf.keras.losses.MeanSquaredError()
    X_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_tensor = tf.convert_to_tensor(y_train.reshape(-1, 1), dtype=tf.float32)
    
    with tf.GradientTape() as tape:
        predictions = model(X_tensor, training=True)
        loss = mse_loss_fn(y_tensor, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    flat_grads = []
    for g in gradients:
        flat = tf.reshape(g, [-1]).numpy()
        flat_grads.extend(flat)
    
    return flat_grads

def add_noise_to_gradients(flat_grads, noise_stddev=1.0):
    return [g + tf.random.normal(shape=g.shape, stddev=noise_stddev) for g in flat_grads]

def reconstruct_grads(flat_grads, model):
    reconstructed = []
    idx = 0
    for var in model.trainable_variables:
        shape = var.shape
        size = np.prod(shape)
        chunk = flat_grads[idx:idx + size]
        tensor = tf.convert_to_tensor(np.array(chunk, dtype=np.float32).reshape(shape))
        reconstructed.append(tensor)
        idx += size
    return reconstructed

def apply_gradients(model, avg_grads):
    reset_seeds()
    optimizer = tf.keras.optimizers.Adam()
    optimizer.apply_gradients(zip(avg_grads, model.trainable_variables))

def fine_tune(model, X_train, y_train):
    reset_seeds()
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

In [4]:
p = 1009 # prime number
g = 11 # generator
x = 5 # private key
h = power(g, x, p) # public key (p, g, h)

clients = [f"2_{i}_Client_Data_" for i in range(1, 10)]
rounds = [
    "2010_2", "2010_3", "2010_4", "2010_5", "2010_6", "2010_7", "2010_8", "2010_9", "2010_10", "2010_11", "2010_12",
    "2011_1", "2011_2", "2011_3", "2011_4", "2011_5", "2011_6", "2011_7", "2011_8", "2011_9", "2011_10", "2011_11", "2011_12",
    "2012_1", "2012_2", "2012_3", "2012_4", "2012_5", "2012_6", "2012_7", "2012_8", 
    #"2012_9", "2012_10"
    ]

client_models = {}

client_r2_history = {i: [] for i in range(9)}

In [5]:
import time

start_time = time.time()

print("Round 2010_2")
train_data = {}
for idx, client in enumerate(clients):
    path = client + rounds[0] + ".csv"
    df = pd.read_csv(path)
    X_train, X_test, y_train, y_test = preprocess_dataset(df, seed=42)
    model, r2 = train_local(X_train, X_test, y_train, y_test)
    client_r2_history[idx].append(r2)
    print(f"R² of Client_{idx+1} ({rounds[0]}): {r2:.4f}")
    train_data[idx] = (X_train, X_test, y_train, y_test)
    client_models[idx] = model

Round 2010_2
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
R² of Client_1 (2010_2): 0.9999
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R² of Client_2 (2010_2): 0.0200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R² of Client_3 (2010_2): 0.9997
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
R² of Client_4 (2010_2): 0.9996
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
R² of Client_5 (2010_2): 0.7836
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
R² of Client_6 (2010_2): 0.9994
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R² of Client_7 (2010_2): 0.9998
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
R² of Client_8 (2010_2): 0.9994
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
R² of Client_9 (2010_2): 0.9995


In [6]:
import time
from phe import paillier

# --- Key generation ---
p = 10089886811898868001
q = 10092003300140014003

n = p * q
public_key = paillier.PaillierPublicKey(n)
private_key = paillier.PaillierPrivateKey(public_key, p, q)

# --- Step 1: Scale and encrypt gradients ---
encrypted_grads_list = []
encryption_time = 0.0

for i, (idx, model) in enumerate(client_models.items()):
    X_train, _, y_train, _ = train_data[idx]
    grad = get_gradients_and_flatten(model, X_train, y_train)

    grad = [int(round(x * 1_000_000)) for x in grad]  # scale to int
    print(f"Grad {i}:\n{grad}")

    start_enc = time.perf_counter()
    enc_grad = [public_key.encrypt(x) for x in grad]
    encryption_time += time.perf_counter() - start_enc

    encrypted_grads_list.append(enc_grad)

# --- Step 2: Aggregate encrypted gradients ---
num_clients = len(encrypted_grads_list)
enc_sum_grad = []

for i in range(len(encrypted_grads_list[0])):
    sum_enc = encrypted_grads_list[0][i]
    for j in range(1, num_clients):
        sum_enc += encrypted_grads_list[j][i]
    enc_sum_grad.append(sum_enc)

# --- Step 3: Decrypt summed gradient ---
decryption_time = 0.0
start_dec = time.perf_counter()
grad_sums = [private_key.decrypt(c) / 1_000_000 for c in enc_sum_grad]
decryption_time += time.perf_counter() - start_dec

print("Sum of Gradients:", grad_sums)

# --- Step 4: Average ---
grads_avg = [round(val / num_clients, 6) for val in grad_sums]
print("Average of Gradients:", grads_avg)

# --- Timing Results ---
print(f"Total encryption time: {encryption_time:.4f} seconds")
print(f"Total decryption time: {decryption_time:.4f} seconds")

Grad 0:
[-124043213, -12660294, -346210419, 0, 0, 0, -227913498, 199307602, -344809387, 0, -342722595, -350888733, -115159660, -9506560547, -565449707, -1623532, 211443100, -482686188, 0, 0, 0, 199359772, 0, 0, 0, 0, 0, -194833679, 160492157, 0, 219608414, 0, 0, 0, 0, 343291901, 0, 0, 0, -182670547, -496260406, 0, 0, 160821945, 0, -11707053711, 0, -4994146484, 0, -600630798, 0, 0, -5672741, 0, -5627257324, 0, 0, -186006485, 0, 0, 0, 258376129, 611496826, 0, 0, 105977020, 0, 330403442, 0, 0, 0, 0, 309689941, -448555542, 0, 0, 0, 0, 0, 225309875, 0, 334858826, 0, 473792419, -9771427734, -6460286133, 0, 0, 305209503, 430681213, 0, 0, -146387924, 0, 0, 148696991, -179765472, -5943419434, 291036804, 0, 4883692871, 683571655, 13549728516, 0, 0, 0, 8839670898, -7775488281, 13519633789, 0, 13377932617, 13658075195, 4486717773, -257281312500, 22043632812, 31583099, -8173721191, 18851343750, 0, 0, 0, -7791199219, 0, 0, 0, 0, 0, 7524835938, -6254746582, 0, -8546186523, 0, 0, 0, 0, -13380316406, 0

In [7]:
# =========================
# Ciphertext size analysis
# =========================

def get_ciphertext_bitlength(ctxt):
    return ctxt.ciphertext().bit_length()

# Size from each client to server (in bits)
client_ciphertext_bits = []
for i, enc_grad in enumerate(encrypted_grads_list):
    total_bits = sum(get_ciphertext_bitlength(c) for c in enc_grad)
    client_ciphertext_bits.append(total_bits)
    print(f"Total size from Client {i} to Server: {total_bits} bits")

# Size from server to clients (summed ciphertexts)
server_ciphertext_bits = sum(get_ciphertext_bitlength(c) for c in enc_sum_grad)
print(f"Total size from Server to Clients (summed ciphertexts): {server_ciphertext_bits} bits")

Total size from Client 0 to Server: 1459210 bits
Total size from Client 1 to Server: 1459592 bits
Total size from Client 2 to Server: 1459382 bits
Total size from Client 3 to Server: 1459731 bits
Total size from Client 4 to Server: 1459321 bits
Total size from Client 5 to Server: 1459654 bits
Total size from Client 6 to Server: 1459595 bits
Total size from Client 7 to Server: 1459407 bits
Total size from Client 8 to Server: 1459521 bits
Total size from Server to Clients (summed ciphertexts): 1459523 bits


In [ ]:
import time
import pandas as pd
from phe import paillier

# --- Initial Setup ---
p = 10089886811898868001
q = 10092003300140014003

n = p * q
public_key = paillier.PaillierPublicKey(n)
private_key = paillier.PaillierPrivateKey(public_key, p, q)

# Initial model training was done before this loop (assumed)
# grads_avg from round 0 is already available

# Begin iterative secure training
start_time = time.time()

for round_id in rounds[1:]:
    print(f"\nRound ({round_id})")
    new_train_data = {}
    new_client_models = {}
    encrypted_grads_list = []
    encryption_time = 0.0

    round_index = rounds.index(round_id)

    # --- Step 1: Client-side training and encryption ---
    for idx, client in enumerate(clients):
        # Combine all round data up to current
        dfs = []
        for r in rounds[:round_index + 1]:
            path = f"{client}{r}.csv"
            df_ind = pd.read_csv(path)
            dfs.append(df_ind)
        df = pd.concat(dfs, ignore_index=True)

        X_train, X_test, y_train, y_test = preprocess_dataset(df, seed=42)

        # Build and apply previous averaged gradient
        model = build_mlp(X_train.shape[1])
        model(X_train[:1])  # model initialization
        avg_grads_tensor = reconstruct_grads(grads_avg, model)
        apply_gradients(model, avg_grads_tensor)

        # Fine-tune with client data
        fine_tune(model, X_train, y_train)

        # Save model and data for next step
        new_train_data[idx] = (X_train, X_test, y_train, y_test)
        new_client_models[idx] = model

        # --- Compute and encrypt gradients ---
        grad = get_gradients_and_flatten(model, X_train, y_train)
        grad = [int(round(x * 1_000_000)) for x in grad]

        start_enc = time.perf_counter()
        enc_grad = [public_key.encrypt(x) for x in grad]
        encryption_time += time.perf_counter() - start_enc

        encrypted_grads_list.append(enc_grad)

        # Evaluate client performance
        r2 = r2_score(y_test, model.predict(X_test))
        client_r2_history[idx].append(r2)
        print(f"R² of Client_{idx+1} ({round_id}): {r2:.4f}")

    # --- Step 2: Aggregate encrypted gradients ---
    num_clients = len(encrypted_grads_list)
    enc_sum_grad = []

    for i in range(len(encrypted_grads_list[0])):
        sum_enc = encrypted_grads_list[0][i]
        for j in range(1, num_clients):
            sum_enc += encrypted_grads_list[j][i]
        enc_sum_grad.append(sum_enc)

    # --- Step 3: Decrypt summed gradient ---
    decryption_time = 0.0
    start_dec = time.perf_counter()
    grad_sums = [private_key.decrypt(c) / 1_000_000 for c in enc_sum_grad]
    decryption_time += time.perf_counter() - start_dec

    # --- Step 4: Average the gradients ---
    grads_avg = [round(val / num_clients, 6) for val in grad_sums]
    print("Average of Gradients:", grads_avg)

    # --- Timing ---
    print(f"Total encryption time: {encryption_time:.4f} seconds")
    print(f"Total decryption time: {decryption_time:.4f} seconds")

    # Update for next round
    train_data = new_train_data
    client_models = new_client_models

end_time = time.time()
print(f"\nSecure Federated Training completed in {(end_time - start_time):.2f} seconds")



Round (2010_3)
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R² of Client_1 (2010_3): 0.1490
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
R² of Client_2 (2010_3): 0.0025
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R² of Client_3 (2010_3): 0.1501
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R² of Client_4 (2010_3): 0.0658
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
R² of Client_5 (2010_3): 0.3544
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R² of Client_6 (2010_3): 0.2250
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
R² of Client_7 (2010_3): 0.2080
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R² of Client_8 (2010_3): 0.2589
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R² of Client_9 (2010_3): 0.2145
Average of Gradients: [-2822.188911, -3943.546807, -2982.322401, 0.0, 0.0, 0.0, -316.099362, -628.902886, -3173.968926, 0.0, -2289.095215, -658.689786, -1430.617499, -4752.575182, -1653.626967, -339.473823, -2273.215481, -3114.09786, 0.0, 3461.690294, 0.0, -668.253947, 0.0, 0.0, 0.0, 0.0, 0.0, 256.408298, -843.934353, 0.0, -1224.927775, 0.0, 0